In [ ]:
# %pip install nbformat

Note: you may need to restart the kernel to use updated packages.


In [2]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [3]:
%run ml_project_normalized.ipynb

Note: you may need to restart the kernel to use updated packages.


/u/nneoma/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
Done running ml_project.ipynb.


In [1]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE, SMOTENC

/u/nneoma/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC

def train_SVM_classifier(X_train, y_train, X_test):

    pipeline = Pipeline([
        ('smote', SMOTENC(categorical_features=[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], random_state=42)),# 'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'is_mix', 'primary_color'], random_state=42)),
        ('freq', FunctionTransformer(apply_freq_encode, validate=False)),
        ('onehot', OneHotEncoder( handle_unknown='ignore', sparse_output=True)),
        ('linsvc', LinearSVC(class_weight='balanced', max_iter=10000, dual=False))
    ])

    # Set up parameter distributions for XGBoost.
    param_distributions = {
    'linsvc__C': [0.01, 0.1, 1, 10],
    'linsvc__loss': ['squared_hinge'],  # 'hinge' is okay too, but 'squared_hinge' usually works better
    'linsvc__penalty': ['l2'],  # 'l1' can only be used with dual=False and 'liblinear' solver
    'linsvc__max_iter': [1000, 5000, 10000],
    'linsvc__multi_class': ['ovr', 'crammer_singer'],  # try both strategies
    }
    # Perform hyperparameter search using RandomizedSearchCV.
    randomized_search = RandomizedSearchCV(
        estimator=pipeline, 
        param_distributions=param_distributions,
        n_iter=1,
        cv=skf, 
        scoring=balanced_acc_scorer, 
        verbose=3,
    )

        
        # Set up parameter distributions for XGBoost.
        param_distributions = {
            "nb__alpha": [0.1, 0.5, 1, 2, 5, 10],
        }
        
        # Perform hyperparameter search using RandomizedSearchCV.
        randomized_search = RandomizedSearchCV(
            estimator=pipeline, 
            param_distributions=param_distributions,
            n_iter=1,
            cv=5, 
            scoring='balanced_accuracy', 
            verbose=3,
        )
        
        randomized_search.fit(X_resampled, y_resampled)
        
        print('Best parameters:', randomized_search.best_params_)
        print('Best cross-validation accuracy:', randomized_search.best_score_)
        
        cv_scores = cross_val_score(randomized_search.best_estimator_, X_resampled, y_resampled, cv=5, verbose=3, scoring='balanced_accuracy')
        print('Generalization Balanced accuracy (via cross_val_score):', cv_scores.mean())

        # Make predictions on the test set using the best estimator.
        test_predictions = randomized_search.predict(X_test)
        
    return randomized_search.best_estimator_, test_predictions


IndentationError: expected an indented block (438336858.py, line 3)